In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Conv2D, Activation, GlobalAveragePooling2D, Dense
import random
from sklearn import metrics

In [2]:
from tensorflow.keras.datasets import fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train = tf.image.per_image_standardization(x_train)
x_test = tf.image.per_image_standardization(x_test)


IMAGE_SIZE = 28
x_test = np.array(x_test).reshape(-1,IMAGE_SIZE,IMAGE_SIZE,1)
x_train = np.array(x_train).reshape(-1,IMAGE_SIZE,IMAGE_SIZE,1)
activation_func = ['relu','sigmoid','tanh','swish','gelu']
filters = [i for i in range(4,128,4)]

In [3]:
class CNN_Model:
  def __init__(self,x_train,y_train,x_test,y_test):
    self.train_x = x_train  
    self.train_y = y_train
    self.test_x = x_test
    self.test_y = y_test
    
    self.activation_function_NC = random.choice(activation_func)
    self.activation_function_RC = random.choice(activation_func)
    self.activation_function_FL = random.choice(activation_func)
  
    self.NC_filters = random.choice(filters)
    self.RC_filters = random.choice(filters)
    self.model = tf.keras.Sequential()
    self.model.add(Conv2D(self.RC_filters,kernel_size=(2,2) ,padding='valid',strides=2,input_shape=x_train.shape[1:])) #Reduction Layer
    self.model.add(Activation(activation=self.activation_function_RC))

    self.model.add(Conv2D(self.NC_filters,(2,2),padding='same')) 
    self.model.add(Activation(activation=self.activation_function_NC))

    self.model.add(Conv2D(self.RC_filters,kernel_size=(2,2) ,padding='valid',strides=2))  #Reduction Layer
    self.model.add(Activation(activation=self.activation_function_RC))

    self.model.add(Conv2D(self.NC_filters,(2,2),padding='same')) 
    self.model.add(Activation(activation=self.activation_function_NC))

    self.model.add(GlobalAveragePooling2D())
    self.model.add(Dense(64))
    self.model.add(Activation(activation=self.activation_function_FL))
    self.model.add(Dense(10))
    self.model.add(Activation("softmax"))

  def get_no_of_parameter(self):
    trainableParams = np.sum([np.prod(v.get_shape()) for v in self.model.trainable_weights])
    nonTrainableParams = np.sum([np.prod(v.get_shape()) for v in self.model.non_trainable_weights])
    totalParams = trainableParams + nonTrainableParams

    return trainableParams, nonTrainableParams, totalParams

  def genome_code(self):
    genome = "RC "+str(self.RC_filters)+" "+str(2)+" "+str(self.activation_function_RC)+"; "+"NC "+str(self.NC_filters)+" "+str(2)+" "+str(self.activation_function_NC)+" ;"+"RC "+str(self.RC_filters)+" "+str(2)+" "+str(self.activation_function_RC)+" ;"+"NC "+str(self.NC_filters)+" "+str(2)+" "+str(self.activation_function_NC)+" ;"+"FL"+" "+str(self.activation_function_FL)
    return genome
  def fit_model(self):
    temp_model = self.model
    temp_model.compile(loss="sparse_categorical_crossentropy",optimizer="adam",metrics=['accuracy'])
    history = temp_model.fit(self.train_x,self.train_y,epochs=8,validation_split=0.2,verbose = 0)
    train_acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    results = temp_model.evaluate(x=self.test_x,y=self.test_y,verbose=0)
    test_acc = results[1]
    return max(train_acc),max(val_acc),test_acc




In [4]:
mod = CNN_Model(x_train=x_train,y_train=y_train,x_test=x_test,y_test=y_test)
trainableParams, nonTrainableParams, totalParams = mod.get_no_of_parameter()
string = mod.genome_code()
print(string)
train_acc,val_acc,test_acc = mod.fit_model()
print("Training Accuracy :- ",train_acc,"Validation Accuracy :- ",val_acc,"Test Accuracy",test_acc)

RC 72 2 gelu; NC 56 2 relu ;RC 72 2 gelu ;NC 56 2 relu ;FL tanh
Training Accuracy :-  0.8957291841506958 Validation Accuracy :-  0.8901666402816772 Test Accuracy 0.8817999958992004


In [5]:
count=1
contents = []
for i in range(100):
  try:
    mod = CNN_Model(x_train=x_train,y_train=y_train,x_test=x_test,y_test=y_test)
    print("Model No:- ",count)
    
    trainableParams, nonTrainableParams, totalParams = mod.get_no_of_parameter()
    string = mod.genome_code()
    print(string)
    train_acc,val_acc,test_acc = mod.fit_model()
    print("Training Accuracy :- ",train_acc,"Validation Accuracy :- ",val_acc,"Test Accuracy",test_acc)
    candidate = {
        "Model No.":count,
        "genome":string,
        "Parameters":totalParams,
        "Training Accuracy":train_acc,
        "Validation Accuracy":val_acc,
        "Test Accuracy":test_acc

    }
    contents.append(candidate)
    count+=1
  except:
    pass

Model No:-  1
RC 52 2 sigmoid; NC 28 2 swish ;RC 52 2 sigmoid ;NC 28 2 swish ;FL relu
Training Accuracy :-  0.8255416750907898 Validation Accuracy :-  0.8328333497047424 Test Accuracy 0.828000009059906
Model No:-  2
RC 16 2 tanh; NC 96 2 sigmoid ;RC 16 2 tanh ;NC 96 2 sigmoid ;FL sigmoid
Training Accuracy :-  0.7875000238418579 Validation Accuracy :-  0.8010833263397217 Test Accuracy 0.7925000190734863
Model No:-  3
RC 76 2 sigmoid; NC 16 2 tanh ;RC 76 2 sigmoid ;NC 16 2 tanh ;FL relu
Training Accuracy :-  0.8108124732971191 Validation Accuracy :-  0.8148333430290222 Test Accuracy 0.7903000116348267
Model No:-  4
RC 96 2 tanh; NC 72 2 swish ;RC 96 2 tanh ;NC 72 2 swish ;FL sigmoid
Training Accuracy :-  0.8808333277702332 Validation Accuracy :-  0.871833324432373 Test Accuracy 0.857200026512146
Model No:-  5
RC 24 2 gelu; NC 100 2 sigmoid ;RC 24 2 gelu ;NC 100 2 sigmoid ;FL tanh
Training Accuracy :-  0.8325625061988831 Validation Accuracy :-  0.831250011920929 Test Accuracy 0.8309000134

In [6]:
import pandas as pd

In [7]:
df = pd.DataFrame(contents)

In [8]:
df

,Model No.,genome,Parameters,Training Accuracy,Validation Accuracy,Test Accuracy
0,1,RC 52 2 sigmoid; NC 28 2 swish ;RC 52 2 sigmoi...,20346.0,0.825542,0.832833,0.8280
1,2,RC 16 2 tanh; NC 96 2 sigmoid ;RC 16 2 tanh ;N...,25578.0,0.787500,0.801083,0.7925
2,3,RC 76 2 sigmoid; NC 16 2 tanh ;RC 76 2 sigmoid...,16818.0,0.810812,0.814833,0.7903
3,4,RC 96 2 tanh; NC 72 2 swish ;RC 96 2 tanh ;NC ...,88986.0,0.880833,0.871833,0.8572
4,5,RC 24 2 gelu; NC 100 2 sigmoid ;RC 24 2 gelu ;...,36258.0,0.832563,0.831250,0.8309
...,...,...,...,...,...,...
95,96,RC 112 2 relu; NC 56 2 gelu ;RC 112 2 relu ;NC...,80346.0,0.889125,0.882333,0.8731
96,97,RC 56 2 gelu; NC 84 2 swish ;RC 56 2 gelu ;NC ...,63042.0,0.900937,0.893250,0.8848
97,98,RC 72 2 relu; NC 56 2 gelu ;RC 72 2 relu ;NC 5...,53226.0,0.892458,0.892000,0.8856
98,99,RC 16 2 gelu; NC 64 2 sigmoid ;RC 16 2 gelu ;N...,17322.0,0.800104,0.806583,0.8029


In [9]:
df.to_csv("genome_Seq_CNN.csv")